In [6]:
import func.datastruct as ds
        

new = ds.Datastruct()
#new.create(path="data/test_dataset.zarr", shape=(512, 512, 512), chunks=(512, 512, 8), mode="r+", engine="zarr")

new.create(path="data/test_dataset.h5", shape=(512, 512, 512), chunks=(512, 512, 8), mode="r+", engine="hdf5")


ModuleNotFoundError: No module named 'func'